In [16]:
from ipynb.fs.full.functions import clean_up, tokenize, stem_and_lemmatize, remove_stopwords
import pandas as pd
import nltk
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [17]:
instagram = pd.read_csv("instagram.csv")
hashtags = pd.read_csv("hashtags.csv")

In [18]:
instagram.head()

,Unnamed: 0,alt,firstComment,likesCount,timestamp,hashtags,post
0,0,Image may contain: one or more people and text,"The best offense is a good defense so here's some healthy tips on how to boast your immunity as work through this Covid-19 scare.\n.\nDon't be scared, just take care.",1,2020-03-12T14:32:48.000Z,[],"the best offense is a good defense so here's some healthy tips on how to boast your immunity as work through this covid-19 scare.\n.\ndon't be scared, just take care."
1,1,Sacred geometry flower of life and goddess artwork,"Nature reflects back to us our own imbalance. When the health of the earth and ecosystems are compromised, our own health is also compromised. We are a part of Nature, and her sickness becomes our sickness ~ her health, our health 🌏 Art: @mimbirose #sacredgeometryart #seedoflife #floweroflife #cyclesofnature",2,2020-03-12T14:22:34.000Z,"['sacredgeometryart', 'seedoflife', 'floweroflife', 'cyclesofnature']","nature reflects back to us our own imbalance. when the health of the earth and ecosystems are compromised, our own health is also compromised. we are a part of nature, and her sickness becomes our sickness ~ her health, our health 🌏 art: @mimbirose"
2,2,Image may contain: 1 person,"Concerning the current outbreak of Covid-19 (Coronavirus), which has now been declared as a pandemic, we must take precautions and safety measures to avoid its spread. Even a simple handshake can cause the transfer of harmful disease-causing bacteria.\n\nA safe way to greet others is through Namaste, a customary and respectful Indian greeting, which is also a part of Atmantan's culture 🙏🏻 #Atmantan #BeTransformed #AtmantanWellnessResort",24,2020-03-12T14:17:54.000Z,"['Atmantan', 'BeTransformed', 'AtmantanWellnessResort']","concerning the current outbreak of covid-19 (coronavirus), which has now been declared as a pandemic, we must take precautions and safety measures to avoid its spread. even a simple handshake can cause the transfer of harmful disease-causing bacteria.\n\na safe way to greet others is through namaste, a customary and respectful indian greeting, which is also a part of atmantan's culture 🙏🏻"
3,3,Image may contain: food,Lunch out today with my daughter😀 jacket potato with tuna and cheese HEA and speedy salad 🥒🍅🥔#slimmingworld#slimmingworldblog#weightlossjourney#weighlossjourney2020#slimmingworldblogger#healthylifestyle#healthyeating#healthyliving#diet#sw#healthylivingjourney,3,2020-03-12T14:17:02.000Z,"['slimmingworld', 'slimmingworldblog', 'weightlossjourney', 'weighlossjourney', 'slimmingworldblogger', 'healthylifestyle', 'healthyeating', 'healthyliving', 'diet', 'sw', 'healthylivingjourney']",lunch out today with my daughter😀 jacket potato with tuna and cheese hea and speedy salad 🥒🍅🥔2020
4,4,"Image may contain: sky, mountain, outdoor, water and nature","Whenever you are faced with fear, you always have a choice.\nFear is something we all have faced and will face it again. It’s inevitable and something needs to change.\n\nDon't let fear win and over power you. Believe that you are strong and capable. ""Forget Everything and Run,’ or, ‘Face Everything and Rise."" The choice is yours.\n\n#choiceisyours #choiceschoices #criticschoice #hardchoices #mindsetchange #mindsetofachampion #selfreflections #selfawareness #mortgageloanofficer #mortgagebanker #mortgageagent #teacherofinstagram #teacherblogger #nurseofinstagram #fitnesshelp #healthylivingjourney #fitmomsclub #getfitforlife #fitmominspire #timefreedom #workfromhomedad #selfemployedlife #goalslayer #workforwhatyouwant #helpinghands #givingback #careforothers #makeadifference #makeanimpact #peoplematter",1,2020-03-12T14:01:39.000Z,"['choiceisyours', 'choiceschoices', 'criticschoice', 'hardchoices', 'mindsetchange', 'mindsetofachampion', 'selfreflections', 'selfawareness', 'mortgageloanofficer', 'mortgagebanker', 'mortgageagent', 'teacherofinstagram', 'teacherblogger', 'nurseofinstagram', 'fitnesshelp', 'healthylivingjourney', 'fitmomsclub', 'getfitfo

In [4]:
hashtags.dropna(inplace=True)

### Preparing Text


In [21]:
instagram.dtypes

Unnamed: 0      int64 
alt             object
firstComment    object
likesCount      int64 
timestamp       object
hashtags        object
post            object
dtype: object

In [23]:
# creating new column where text has been cleaned, tokenized, stemmed and lemmatized

instagram['text_processed'] = instagram['post'].apply(str).apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)


In [24]:
instagram.text_processed.head()

0    [best, offens, good, defens, healthi, tip, boast, immun, work, thi, covid, scare, scare, take, care]                                                                                                                                                                                   
1    [natur, reflect, back, u, imbal, health, earth, ecosystem, compromis, health, also, compromis, part, natur, sick, becom, sick, health, health, art, mimbiros]                                                                                                                          
2    [concern, current, outbreak, covid, coronaviru, ha, declar, pandem, must, take, precaut, safeti, measur, avoid, spread, even, simpl, handshak, caus, transfer, harm, diseas, caus, bacteria, safe, way, greet, namast, customari, respect, indian, greet, also, part, atmantan, cultur]
3    [lunch, today, daughter, jacket, potato, tuna, chees, hea, speedi, salad]                                                                   

#### Prepare bi-grams and tri-grams

In [25]:
from gensim.models import Phrases

In [26]:
tokens = instagram.text_processed.tolist()
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count = 1)
tokens = list(trigram_model[bigram_model[tokens]])

#### Prepare objects for LDA gensim implementation


In [27]:
from gensim import corpora


In [28]:
dictionary_LDA = corpora.Dictionary(tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]

#### Running LDA

In [29]:
from gensim import models
import numpy as np

In [30]:
np.random.seed(123456)
num_topics = 20
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

CPU times: user 22.1 s, sys: 115 ms, total: 22.2 s
Wall time: 22.3 s


#### Quick exploration

In [32]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20):
    print(str(i)+": "+ topic)
    print()

0: 0.017*"also" + 0.017*"thi" + 0.016*"use" + 0.015*"need" + 0.015*"diet" + 0.013*"food" + 0.012*"help" + 0.009*"bodi" + 0.009*"make" + 0.008*"eat" + 0.008*"like" + 0.008*"healthi" + 0.007*"may" + 0.007*"health" + 0.007*"one" + 0.007*"ha" + 0.006*"caus" + 0.006*"includ" + 0.005*"calori" + 0.005*"option"

1: 0.061*"e" + 0.049*"la" + 0.038*"un" + 0.025*"que" + 0.023*"oil" + 0.023*"l" + 0.021*"essenti_oil" + 0.019*"con" + 0.018*"skin" + 0.017*"si" + 0.014*"il" + 0.013*"di" + 0.013*"contact_u" + 0.012*"lo" + 0.011*"mi" + 0.009*"com" + 0.009*"mon" + 0.009*"stir" + 0.009*"strawberri" + 0.008*"hello"

2: 0.035*"recip" + 0.031*"dan" + 0.025*"salad" + 0.023*"cook" + 0.022*"add" + 0.021*"lunch" + 0.020*"veggi" + 0.020*"vegan" + 0.019*"smoothi" + 0.019*"delici" + 0.015*"enjoy" + 0.013*"breakfast" + 0.012*"yang" + 0.012*"mix" + 0.012*"ingredi" + 0.011*"chicken" + 0.011*"yummi" + 0.011*"make" + 0.010*"serv" + 0.010*"c"

3: 0.034*"start" + 0.030*"get" + 0.017*"thi" + 0.016*"day" + 0.015*"time" + 0.0

In [33]:
lda_model[corpus[0]] # the first document is 96% about topic 11. 
# not sure on how to read this

[(7, 0.96345174)]

In [34]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
